# Sleeper Fantasy Football Stats Analysis

Comprehensive statistical analysis of Hoyt Sleeper fantasy football league using DuckDB for local querying and exploration.

This notebook fetches matchup data from Hoyt Sleeper league and computes:
- Overall standings and win/loss records
- Top 10 highest and lowest individual week performances
- Scoring frequency (most high/low scores by week)
- Head-to-head performance metrics
- Average points in wins vs losses


## Setup: Import Libraries and Initialize Database


In [51]:
import requests
import pandas as pd
import duckdb
import json
from datetime import datetime
from typing import Dict, List, Any
import warnings
warnings.filterwarnings('ignore')

# Initialize DuckDB in-memory database
db = duckdb.connect(':memory:')


## Fetch League Data from Sleeper API


In [52]:
# Read league ID
LEAGUE_ID = 1255671107596333056
BASE_URL = "https://api.sleeper.app/v1"

# Fetch league information
league_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}")
league_info = league_response.json()


## Fetch Users, Rosters, and Matchups


In [53]:
# Fetch users and rosters
users_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/users")
users_list = users_response.json()
users_dict = {user['user_id']: user for user in users_list}

rosters_response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/rosters")
rosters_list = rosters_response.json()
rosters_dict = {roster['roster_id']: roster for roster in rosters_list}


In [54]:
# Fetch matchups for all weeks
matchups_by_week = {}

for week in range(1, 22):
    try:
        response = requests.get(f"{BASE_URL}/league/{LEAGUE_ID}/matchups/{week}")
        if response.status_code == 200:
            matchups_list = response.json()
            if matchups_list:
                matchups_by_week[week] = matchups_list
            else:
                break
    except Exception as e:
        break


## Build DuckDB Tables


In [55]:
# Build matchup records
matchup_records = []

for week, matchups in matchups_by_week.items():
    for matchup in matchups:
        roster_id = matchup['roster_id']
        matchup_id = matchup['matchup_id']
        
        roster = rosters_dict.get(roster_id, {})
        owner_id = roster.get('owner_id')
        user = users_dict.get(owner_id, {})
        team_name = user.get('metadata', {}).get('team_name', f'Team_{roster_id}')
        
        points = matchup.get('points', 0)
        
        players = matchup.get('players', [])
        player_scores = {}
        for player_id in players:
            score = matchup.get('players_points', {}).get(player_id, 0)
            if score > 0:
                player_scores[player_id] = score
        
        if player_scores:
            top_player_id = max(player_scores, key=player_scores.get)
            top_player_score = player_scores[top_player_id]
        else:
            top_player_id = None
            top_player_score = 0
        
        matchup_records.append({
            'week': week,
            'roster_id': roster_id,
            'team_name': team_name,
            'matchup_id': matchup_id,
            'points': points,
            'top_player_id': top_player_id,
            'top_player_score': top_player_score,
            'players_json': json.dumps(player_scores)
        })

matchups_df = pd.DataFrame(matchup_records)
db.register('matchups_raw', matchups_df)


## Create Matchups Statistics Table


In [56]:
# Create matchups table with win/loss calculation
db.execute("DROP TABLE IF EXISTS matchups")

opponent_mapping = {}
for week, matchups in matchups_by_week.items():
    for matchup in matchups:
        roster_id = matchup['roster_id']
        matchup_id = matchup['matchup_id']
        
        key = (week, matchup_id, roster_id)
        opponent_mapping[key] = None

for week, matchups in matchups_by_week.items():
    for matchup in matchups:
        roster_id = matchup['roster_id']
        matchup_id = matchup['matchup_id']
        
        for other_matchup in matchups:
            if other_matchup['matchup_id'] == matchup_id and other_matchup['roster_id'] != roster_id:
                opponent_id = other_matchup['roster_id']
                opponent_roster = rosters_dict.get(opponent_id, {})
                opponent_owner_id = opponent_roster.get('owner_id')
                opponent_user = users_dict.get(opponent_owner_id, {})
                opponent_name = opponent_user.get('metadata', {}).get('team_name', f'Team_{opponent_id}')
                
                key = (week, matchup_id, roster_id)
                opponent_mapping[key] = opponent_name
                break

db.execute("""
    CREATE TABLE matchups AS
    SELECT 
        m.week,
        m.roster_id,
        m.team_name,
        m.matchup_id,
        m.points,
        m.top_player_id,
        m.top_player_score,
        m.players_json,
        CASE WHEN m.points > COALESCE(m2.points, 0) THEN 1 ELSE 0 END as won
    FROM matchups_raw m
    LEFT JOIN matchups_raw m2 ON m.week = m2.week 
        AND m.matchup_id = m2.matchup_id 
        AND m.roster_id != m2.roster_id
""")


## Statistical Analysis Results

Complete league statistics for weeks 1-14 (completed weeks only):


In [57]:
# Nothing to display here - just running queries for the final stats
pass


In [58]:
print("\n")
print("╔" + "═" * 98 + "╗")
print("║" + " FANTASY FOOTBALL STATS - WEEKS 1-14 ".center(98) + "║")
print("╚" + "═" * 98 + "╝")

# OVERALL STANDINGS
print("\n📊 OVERALL STANDINGS")
print("─" * 100)
standings = db.execute("""
SELECT 
    team_name,
    SUM(won) as wins,
    COUNT(*) - SUM(won) as losses,
    ROUND(AVG(points), 2) as avg_pts
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY wins DESC, avg_pts DESC
""").df()

for i, (_, row) in enumerate(standings.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['wins'])}-{int(row['losses']):<3}  {row['avg_pts']:>7.2f} avg pts")

# TOP 10 SCORES
print("\n\n🔥 TOP 10 SCORES")
print("─" * 100)
top_10 = db.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY points DESC) as rank,
    team_name,
    week,
    ROUND(points, 2) as points,
    CASE WHEN won = 1 THEN 'W' ELSE 'L' END as result
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
ORDER BY points DESC
LIMIT 10
""").df()

for _, row in top_10.iterrows():
    print(f"  {int(row['rank']):2}. {row['team_name']:<30} Wk {int(row['week']):2}  {row['points']:>7.2f}  [{row['result']}]")

# TOP 10 LOWEST SCORES
print("\n\n❄️  TOP 10 LOWEST SCORES")
print("─" * 100)
low_10 = db.execute("""
SELECT 
    ROW_NUMBER() OVER (ORDER BY points ASC) as rank,
    team_name,
    week,
    ROUND(points, 2) as points,
    CASE WHEN won = 1 THEN 'W' ELSE 'L' END as result
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
ORDER BY points ASC
LIMIT 10
""").df()

for _, row in low_10.iterrows():
    print(f"  {int(row['rank']):2}. {row['team_name']:<30} Wk {int(row['week']):2}  {row['points']:>7.2f}  [{row['result']}]")

# CONSISTENCY
print("\n\n🎯 CONSISTENCY SCORE (Lower = More Consistent)")
print("─" * 100)
consistency = db.execute("""
SELECT 
    team_name,
    ROUND(AVG(points), 2) as avg_pts,
    ROUND(STDDEV_POP(points), 2) as std_dev
FROM matchups
WHERE week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY std_dev ASC
""").df()

for i, (_, row) in enumerate(consistency.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['std_dev']:>6.2f} std dev")

# MOST HIGH SCORES BY WEEK
print("\n\n🏆 MOST HIGH SCORES BY WEEK")
print("─" * 100)
high_freq = db.execute("""
SELECT 
    team_name,
    COUNT(*) as times_highest
FROM (
    SELECT 
        week,
        team_name,
        points,
        ROW_NUMBER() OVER (PARTITION BY week ORDER BY points DESC) as rank
    FROM matchups
    WHERE week >= 1 AND week <= 14 AND points > 0
)
WHERE rank = 1
GROUP BY team_name
ORDER BY times_highest DESC
""").df()

for i, (_, row) in enumerate(high_freq.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['times_highest']):>2} times")

# MOST LOW SCORES BY WEEK
print("\n\n📉 MOST LOW SCORES BY WEEK")
print("─" * 100)
low_freq = db.execute("""
SELECT 
    team_name,
    COUNT(*) as times_lowest
FROM (
    SELECT 
        week,
        team_name,
        points,
        ROW_NUMBER() OVER (PARTITION BY week ORDER BY points ASC) as rank
    FROM matchups
    WHERE week >= 1 AND week <= 14 AND points > 0
)
WHERE rank = 1
GROUP BY team_name
ORDER BY times_lowest DESC
""").df()

for i, (_, row) in enumerate(low_freq.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['times_lowest']):>2} times")

# WINS VS LOWEST OPPONENT
print("\n\n⚡ MOST WINS VS LOWEST-SCORING OPPONENT")
print("─" * 100)
clutch = db.execute("""
WITH opponent_points AS (
    SELECT 
        m1.week,
        m1.matchup_id,
        m1.team_name,
        m1.points as my_points,
        m1.won,
        m2.points as opponent_points
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
),
weekly_min AS (
    SELECT 
        week,
        MIN(opponent_points) as min_opponent_score
    FROM opponent_points
    GROUP BY week
)
SELECT 
    op.team_name,
    COUNT(*) as wins_vs_low_opp,
    ROUND(AVG(op.opponent_points), 2) as avg_opp_score
FROM opponent_points op
JOIN weekly_min wm ON op.week = wm.week AND op.opponent_points = wm.min_opponent_score
WHERE op.won = 1
GROUP BY op.team_name
ORDER BY wins_vs_low_opp DESC
""").df()

for i, (_, row) in enumerate(clutch.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['wins_vs_low_opp']):>2} wins")

# LOSSES VS HIGHEST OPPONENT
print("\n\n😰 MOST LOSSES VS HIGHEST-SCORING OPPONENT")
print("─" * 100)
tough = db.execute("""
WITH opponent_points AS (
    SELECT 
        m1.week,
        m1.matchup_id,
        m1.team_name,
        m1.points as my_points,
        m1.won,
        m2.points as opponent_points
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
),
weekly_max AS (
    SELECT 
        week,
        MAX(opponent_points) as max_opponent_score
    FROM opponent_points
    GROUP BY week
)
SELECT 
    op.team_name,
    COUNT(*) as losses_vs_high_opp,
    ROUND(AVG(op.opponent_points), 2) as avg_opp_score
FROM opponent_points op
JOIN weekly_max wm ON op.week = wm.week AND op.opponent_points = wm.max_opponent_score
WHERE op.won = 0
GROUP BY op.team_name
ORDER BY losses_vs_high_opp DESC
""").df()

for i, (_, row) in enumerate(tough.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {int(row['losses_vs_high_opp']):>2} losses")

# AVG POINTS IN WINS
print("\n\n💪 AVG POINTS IN WINS")
print("─" * 100)
wins = db.execute("""
SELECT 
    team_name,
    ROUND(AVG(points), 2) as avg_pts_in_win
FROM matchups
WHERE won = 1 AND week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY avg_pts_in_win DESC
""").df()

for i, (_, row) in enumerate(wins.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['avg_pts_in_win']:>7.2f} avg")

# AVG POINTS IN LOSSES
print("\n\n😔 AVG POINTS IN LOSSES")
print("─" * 100)
losses = db.execute("""
SELECT 
    team_name,
    ROUND(AVG(points), 2) as avg_pts_in_loss
FROM matchups
WHERE won = 0 AND week >= 1 AND week <= 14 AND points > 0
GROUP BY team_name
ORDER BY avg_pts_in_loss DESC
""").df()

for i, (_, row) in enumerate(losses.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['avg_pts_in_loss']:>7.2f} avg")

# MOST DOMINANT WINS
print("\n\n🏅 MOST DOMINANT WINS (Biggest Point Margins)")
print("─" * 100)
dominant = db.execute("""
SELECT 
    m1.team_name,
    m1.week,
    ROUND(m1.points - m2.points, 2) as margin,
    ROUND(m1.points, 2) as pts_for,
    ROUND(m2.points, 2) as pts_against
FROM matchups m1
JOIN matchups m2 ON m1.week = m2.week 
    AND m1.matchup_id = m2.matchup_id 
    AND m1.roster_id != m2.roster_id
WHERE m1.won = 1 AND m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
ORDER BY margin DESC
LIMIT 10
""").df()

for idx, (_, row) in enumerate(dominant.iterrows(), 1):
    print(f"  {idx:2}. {row['team_name']:<30} Wk {int(row['week']):2}  +{row['margin']:>6.2f}")

# CLOSEST MATCHUPS
print("\n\n🤝 CLOSEST MATCHUPS (Smallest Margins)")
print("─" * 100)
closest = db.execute("""
SELECT 
    m1.team_name,
    m2.team_name as opponent,
    m1.week,
    ROUND(ABS(m1.points - m2.points), 2) as margin,
    CASE WHEN m1.points > m2.points THEN 'W' ELSE 'L' END as result
FROM matchups m1
JOIN matchups m2 ON m1.week = m2.week 
    AND m1.matchup_id = m2.matchup_id 
    AND m1.roster_id != m2.roster_id
WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
ORDER BY margin ASC
LIMIT 10
""").df()

for idx, (_, row) in enumerate(closest.iterrows(), 1):
    print(f"  {idx:2}. {row['team_name']:<25} vs {row['opponent']:<25} {row['result']} by {row['margin']:>5.2f}")

# LUCKY/UNLUCKY RECORD
print("\n\n🍀 LUCKY/UNLUCKY RECORD (Actual vs Expected)")
print("─" * 100)
lucky = db.execute("""
WITH team_totals AS (
    SELECT 
        team_name,
        SUM(won) as actual_wins,
        COUNT(*) - SUM(won) as actual_losses
    FROM matchups
    WHERE week >= 1 AND week <= 14 AND points > 0
    GROUP BY team_name
),
expected_wins AS (
    SELECT 
        m1.team_name,
        COUNT(*) as expected_wins
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
        AND m1.points > m2.points
    GROUP BY m1.team_name
)
SELECT 
    t.team_name,
    t.actual_wins,
    t.actual_losses,
    e.expected_wins,
    (t.actual_wins - e.expected_wins) as luck_factor
FROM team_totals t
LEFT JOIN expected_wins e ON t.team_name = e.team_name
ORDER BY luck_factor DESC
""").df()

for _, row in lucky.iterrows():
    actual = f"{int(row['actual_wins'])}-{int(row['actual_losses'])}"
    expected = f"{int(row['expected_wins'])}-{14 - int(row['expected_wins'])}"
    luck = int(row['luck_factor'])
    emoji = "😍" if luck > 0 else "😢" if luck < 0 else "🤷"
    diff = f"+{luck}" if luck > 0 else f"{luck}"
    print(f"  {row['team_name']:<30} {actual:<6} (exp: {expected:<6}) {emoji} {diff:>3}")

# WEEK-TO-WEEK VOLATILITY
print("\n\n📈 WEEK-TO-WEEK VOLATILITY (Biggest Swings)")
print("─" * 100)
volatility = db.execute("""
WITH week_pairs AS (
    SELECT 
        t1.team_name,
        ROUND(ABS(t1.points - t2.points), 2) as swing
    FROM matchups t1
    JOIN matchups t2 ON t1.team_name = t2.team_name 
        AND t1.week + 1 = t2.week
    WHERE t1.week >= 1 AND t1.week < 14 AND t1.points > 0 AND t2.points > 0
)
SELECT 
    team_name,
    ROUND(AVG(swing), 2) as avg_swing,
    MAX(swing) as biggest_swing
FROM week_pairs
GROUP BY team_name
ORDER BY biggest_swing DESC
""").df()

for i, (_, row) in enumerate(volatility.iterrows(), 1):
    print(f"  {i:2}. {row['team_name']:<30} {row['biggest_swing']:>6.2f}")

# HEAD-TO-HEAD RECORDS
print("\n\n🎲 HEAD-TO-HEAD RECORDS")
print("─" * 100)
h2h = db.execute("""
WITH h2h_records AS (
    SELECT 
        m1.team_name,
        m2.team_name as opponent,
        SUM(CASE WHEN m1.points > m2.points THEN 1 ELSE 0 END) as wins,
        COUNT(*) as games
    FROM matchups m1
    JOIN matchups m2 ON m1.week = m2.week 
        AND m1.matchup_id = m2.matchup_id 
        AND m1.roster_id != m2.roster_id
    WHERE m1.week >= 1 AND m1.week <= 14 AND m1.points > 0
    GROUP BY m1.team_name, m2.team_name
    ORDER BY m1.team_name, wins DESC
)
SELECT * FROM h2h_records
""").df()

current_team = None
for _, row in h2h.iterrows():
    if row['team_name'] != current_team:
        if current_team is not None:
            print()
        current_team = row['team_name']
        print(f"  {current_team}")
    losses = int(row['games']) - int(row['wins'])
    print(f"    vs {row['opponent']:<28} {int(row['wins'])}-{losses}")

print("\n╔" + "═" * 98 + "╗")
print("║" + " END OF STATS ".center(98) + "║")
print("╚" + "═" * 98 + "╝\n")




╔══════════════════════════════════════════════════════════════════════════════════════════════════╗
║                               FANTASY FOOTBALL STATS - WEEKS 1-14                                ║
╚══════════════════════════════════════════════════════════════════════════════════════════════════╝

📊 OVERALL STANDINGS
────────────────────────────────────────────────────────────────────────────────────────────────────
   1. Egbuka Matata                  11-3     140.46 avg pts
   2. Jefferson Starship             9-5     127.11 avg pts
   3. 🙌🏼 Praise Jah 🙌🏼               8-6     120.36 avg pts
   4. El Travador                    7-7     129.66 avg pts
   5. Luke at my Big Dicker          7-7     127.92 avg pts
   6. Spicy Bijan Mustard            7-7     126.21 avg pts
   7. Bridezilla                     7-7     115.74 avg pts
   8. Super Weenie Hut Jr            6-8     113.19 avg pts
   9. Kenergy                        4-10    122.97 avg pts
  10. Kittle Me This            